## Installs

In [ ]:
#install langchain 
!pip -q install langchain openai==0.27.0  tiktoken 

In [ ]:
# install bark (make sure you have torch>=2 for much faster flash-attention)
!pip install git+https://github.com/suno-ai/bark.git

In [1]:
!pip install prompt-optimizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.8 MB/s eta 0:00:00
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=2a9f8de64800862aa94cbcb3cbc99f320f793e645cc2d17d04759af8c7ab8aa3
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b

In [ ]:
!pip install --upgrade tiktoken

# Inputs



In [17]:
#ask for API keys and input text

ApiKey = input('what is your API key?')

text = input('copy and paste a page that you wanted summarized into a soundbite here')

what is your API key?1234
copy and paste a page that you wanted summarized into a soundbite hereThe 2023 Indian Premier League (also known as Tata IPL 2023 for sponsorship reasons and sometimes referred to as IPL 2023 or IPL 16) was the 16th season of the Indian Premier League, a franchise Twenty20 cricket league in India. It is organised by the Board of Control for Cricket in India.[1]  In the final, Chennai Super Kings defeated Gujarat Titans, by five wickets (DLS method) to win their record fifth league title.[2]


# trimming and counting tokens

In [18]:
#new prompt optimizier for v0.1.0
from prompt_optimizer.poptim import EntropyOptim


In [19]:
#semantically trims prompt with minimal loss of semantic value before trimming 
prompt = text
p_optimizer = EntropyOptim(verbose=True, p=0.1) #reccomended: p = 0.1 represents an 11% reduction in tokens while reducing logiQA accuracy by only 6%; other options such as p = 0.05, 0.25, 0.5 are available(0.5 not reccomended)
optimized_prompt = p_optimizer(prompt)
optimized_prompt_output = optimized_prompt['content']
text = optimized_prompt_output
print(text)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-06-07 15:53:33,580 - prompt_optimizer.poptim.logger - INFO - Prompt Data Before: The 2023 Indian Premier League (also known as Tata IPL 2023 for sponsorship reasons and sometimes referred to as IPL 2023 or IPL 16) was the 16th season of the Indian Premier League, a franchise Twenty20 cricket league in India. It is organised by the Board of Control for Cricket in

The 2023 Indian Premier League ( also known Tata IPL 2023 for sponsorship reasons and sometimes referred to as IPL 2023 or IPL 16 ) was the 16th season Indian Premier, a franchise Twenty20 cricket league in India. is organised the Board Control for Cricket in. [ 1 ] In the final, Super Kings defeated Gujarat Titans, by five wickets ( DLS method ) win their record fifth league title. [ 2 ]


In [20]:
#prompt trimming inspired by GPTrim
import re
from typing import Optional, List

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:


ARTICLES_PREPOSITIONS = {
    "english": ['the', 'a', 'an', 'in', 'on', 'at', 'for', 'to', 'of']
}

NEGATION_WORDS = {
    "english": [
        'no',
        'nor',
        'not',
        'don',
        "dont",
        'ain',
        'aren',
        "arent",
        'couldn',
        "couldnt",
        'didn',
        "didnt",
        'doesn',
        "doesnt",
        'hadn',
        "hadnt",
        'hasn',
        "hasnt",
        'haven',
        "havent",
        'isn',
        "isnt",
        'mightn',
        "mightnt",
        'mustn',
        "mustnt",
        'needn',
        "neednt",
        'shan',
        "shant",
        'shouldn',
        "shouldnt",
        'wasn',
        "wasnt",
        'weren',
        "werent",
        'won',
        "wont",
        'wouldn',
        "wouldnt",
    ],
}

PUNCTUATION = [".", ",", "'", '"', "!", "?", ";", ":", "-", "(", ")","[","]","{","}"] #now also removes parantheses, brackets, and braces

def trim(
    text: str, stemmer: Optional[str] = None, language: str = "english", remove_spaces: bool = True,
        remove_stopwords: bool = True, remove_punctuation: bool = True) -> str:

    if language not in stopwords.fileids():
        raise ValueError("Unsupported language")

    accepted_stemmers = ("snowball", "porter", "lancaster")
    if stemmer and stemmer not in accepted_stemmers:
        raise ValueError("Stemmer must be one of", accepted_stemmers)

    # merge contractions
    text: str = text.replace("'", "").replace("’", "")

    # tokenize words, keep uppercase
    tokenized: List = nltk.word_tokenize(text)

    if remove_punctuation:
        tokenized = [word for word in tokenized if word not in PUNCTUATION]

    if remove_stopwords:
        nltk_stopwords = stopwords.words(language)
        words_to_exclude = set(
            nltk_stopwords + ARTICLES_PREPOSITIONS.get(language, [])
        ) - set(NEGATION_WORDS.get(language, []))

        tokenized = [word for word in tokenized if word.lower() not in words_to_exclude]

    words = tokenized

    if stemmer:
        if stemmer == "porter":
            stemmer = PorterStemmer()
        elif stemmer == "snowball":
            stemmer = SnowballStemmer(language)
        elif stemmer == "lancaster":
            stemmer = LancasterStemmer()
        words = [stemmer.stem(word) for word in tokenized]

        # restore title_case and uppercase after stemming
        case_restored = []
        for i, word in enumerate(words):
            if tokenized[i].istitle():
                word = word.title()
            elif tokenized[i].isupper():
                word = word.upper()
            case_restored.append(word)

        words = case_restored
    #delete the last period 
    words2 = words.pop()

    # remove spaces
    #join_str = "" if remove_spaces else " "
    #trimmed: str = join_str.join(words).strip()
    #if not remove_punctuation:
        # this is a hack to remove spaces before punctuation
        #trimmed = re.sub(r"\s([?.!,:;])", r"\1", trimmed)
        
    return ' '.join(words)

In [24]:
#text is from the input above
trimmed_text = trim(text)
print(trimmed_text)

2023 Indian Premier League also known Tata IPL 2023 sponsorship reasons sometimes referred IPL 2023 IPL 16 16th season Indian Premier franchise Twenty20 cricket league India organised Board Control Cricket 1 final Super Kings defeated Gujarat Titans five wickets DLS method win record fifth league title


In [ ]:
import tiktoken

In [ ]:
#see if there is a valid number of tokens 
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [ ]:
numTokens = len(encoding.encode(trimmed_text))

In [ ]:
#just for comparison with original 
len(encoding.encode(text))

In [ ]:
if numTokens >= 4096:
  print('TOO MANY TOKENS')
else:
  print('Go ahead')

# Setting Up Langchain and GPT-3.5 turbo

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip show langchain

# Text Summarization


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chatGPT = ChatOpenAI(temperature=0)

In [ ]:
messages = [
    SystemMessage(content="You are an expert at making strong factual summarizations.\
     Take the article submitted by the user and produce a factual useful summary"),
    HumanMessage(content=trimmed_text)
]
responses = chatGPT(messages)

In [ ]:
#summarizing the text (something with langchain) 
summarizedText = responses
print(summarizedText)



# Conversion to realistic text to speech and read out 

(can take a while) 

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
#long form generation

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from IPython.display import Audio
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE



In [ ]:
preload_models()

In [ ]:
script = summarizedText.replace("\n", " ").strip()
sentences = nltk.sent_tokenize(script)
GEN_TEMP = 0.6
SPEAKER = "v2/en_speaker_6"
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]
    Audio(np.concatenate(pieces), rate=SAMPLE_RATE)